In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
# Read CSV
sleep_df = pd.read_csv("Sleep_Efficiency_Updated.csv")
sleep_df.head()

,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,1,65,Female,06/03/2021 01:00,06/03/2021 07:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
1,2,69,Male,05/12/2021 02:00,05/12/2021 09:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,25/05/2021 21:30,25/05/2021 05:30,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0
3,4,40,Female,03/11/2021 02:30,03/11/2021 08:30,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0
4,5,57,Male,13/03/2021 01:00,13/03/2021 09:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0


In [3]:
# Drop non-beneficial columns
sleep_df = sleep_df.drop(columns=['ID','Bedtime', 'Wakeup time'])

In [4]:
# Drop null values
sleep_df = sleep_df.dropna()

In [5]:
sleep_df.nunique()

Age                       60
Gender                     2
Sleep duration             9
Sleep efficiency          50
REM sleep percentage      13
Deep sleep percentage     28
Light sleep percentage    28
Awakenings                 5
Caffeine consumption       6
Alcohol consumption        6
Smoking status             2
Exercise frequency         6
dtype: int64

In [6]:
sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 528 entries, 0 to 617
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     528 non-null    int64  
 1   Gender                  528 non-null    object 
 2   Sleep duration          528 non-null    float64
 3   Sleep efficiency        528 non-null    float64
 4   REM sleep percentage    528 non-null    int64  
 5   Deep sleep percentage   528 non-null    int64  
 6   Light sleep percentage  528 non-null    int64  
 7   Awakenings              528 non-null    float64
 8   Caffeine consumption    528 non-null    float64
 9   Alcohol consumption     528 non-null    float64
 10  Smoking status          528 non-null    object 
 11  Exercise frequency      528 non-null    float64
dtypes: float64(6), int64(4), object(2)
memory usage: 53.6+ KB


In [7]:
# Get Dummies
sleep_df = pd.get_dummies(sleep_df,dtype=float)

In [8]:
sleep_df.head()

,Age,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Exercise frequency,Gender_Female,Gender_Male,Smoking status_No,Smoking status_Yes
0,65,6.0,0.88,18,70,12,0.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0
1,69,7.0,0.66,19,28,53,3.0,0.0,3.0,3.0,0.0,1.0,0.0,1.0
2,40,8.0,0.89,20,70,10,1.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0
3,40,6.0,0.51,23,25,52,3.0,50.0,5.0,1.0,1.0,0.0,0.0,1.0
4,57,8.0,0.76,27,55,18,3.0,0.0,3.0,3.0,0.0,1.0,1.0,0.0


In [9]:
# Convert Sleep efficiency values > .85 equal to 1
sleep_df['Sleep efficiency'] = sleep_df['Sleep efficiency'].apply(lambda x: 1 if x > 0.85 else 0)

In [10]:
sleep_df

,Age,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Exercise frequency,Gender_Female,Gender_Male,Smoking status_No,Smoking status_Yes
0,65,6.0,1,18,70,12,0.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0
1,69,7.0,0,19,28,53,3.0,0.0,3.0,3.0,0.0,1.0,0.0,1.0
2,40,8.0,1,20,70,10,1.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0
3,40,6.0,0,23,25,52,3.0,50.0,5.0,1.0,1.0,0.0,0.0,1.0
4,57,8.0,0,27,55,18,3.0,0.0,3.0,3.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,61,7.0,0,23,23,54,2.0,50.0,5.0,0.0,1.0,0.0,1.0,0.0
614,30,8.0,1,22,63,15,1.0,75.0,0.0,2.0,1.0,0.0,0.0,1.0
615,48,6.0,0,24,60,16,4.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0
616,32,5.0,1,20,65,15,1.0,25.0,1.0,0.0,1.0,0.0,1.0,0.0


## Alcohol consumption Vs Sleep efficiency

In [11]:
y = sleep_df['Sleep efficiency'].values

X = sleep_df.drop(['Alcohol consumption','Sleep efficiency'], axis=1).values

In [12]:
# Split data to training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [13]:
# Standard Scaler instance
scaler = StandardScaler()

# Fit Standard Scaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Neural Net model, nodes, layers

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 104       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 155 (620.00 Byte)
Trainable params: 155 (620.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50


13/13 [==============================] - 2s 4ms/step - loss: 0.5820 - accuracy: 0.6338
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 0.5610 - accuracy: 0.6641
Epoch 3/50
13/13 [==============================] - 0s 4ms/step - loss: 0.5424 - accuracy: 0.6970
Epoch 4/50
13/13 [==============================] - 0s 4ms/step - loss: 0.5261 - accuracy: 0.7475
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.5112 - accuracy: 0.7652
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.4975 - accuracy: 0.7778
Epoch 7/50
13/13 [==============================] - 0s 4ms/step - loss: 0.4847 - accuracy: 0.7904
Epoch 8/50
13/13 [==============================] - 0s 3ms/step - loss: 0.4718 - accuracy: 0.8106
Epoch 9/50
13/13 [==============================] - 0s 5ms/step - loss: 0.4588 - accuracy: 0.8157
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 0.4462 - accuracy: 0.8232
Epoch 11/50
13/13

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5/5 - 0s - loss: 0.2728 - accuracy: 0.8712 - 262ms/epoch - 52ms/step
Loss: 0.2728332579135895, Accuracy: 0.8712121248245239


## Caffeine consumption Vs Sleep efficiency

In [18]:
y = sleep_df['Sleep efficiency'].values

X = sleep_df.drop(['Sleep efficiency','Caffeine consumption'], axis=1).values

In [19]:
# Split data to training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [20]:
# Standard Scaler instance
scaler = StandardScaler()

# Fit Standard Scaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Neural Net model, nodes, layers

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 104       
                                                                 
 dense_4 (Dense)             (None, 5)                 45        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 155 (620.00 Byte)
Trainable params: 155 (620.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
13/13 [==============================] - 1s 3ms/step - loss: 0.8120 - accuracy: 0.4621
Epoch 2/50
13/13 [==============================] - 0s 4ms/step - loss: 0.7668 - accuracy: 0.4949
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7316 - accuracy: 0.5177
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7006 - accuracy: 0.5657
Epoch 5/50
13/13 [==============================] - 0s 4ms/step - loss: 0.6743 - accuracy: 0.5909
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6526 - accuracy: 0.6187
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6336 - accuracy: 0.6364
Epoch 8/50
13/13 [==============================] - 0s 4ms/step - loss: 0.6149 - accuracy: 0.6641
Epoch 9/50
13/13 [==============================] - 0s 4ms/step - loss: 0.5977 - accuracy: 0.6894
Epoch 10/50
13/13 [==============================] - 0s 4ms/step - loss: 0.5809 - accuracy: 0.7273
Epoch 11/50
13/13 [

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5/5 - 0s - loss: 0.2789 - accuracy: 0.8636 - 182ms/epoch - 36ms/step
Loss: 0.278888463973999, Accuracy: 0.8636363744735718


## Smoking status Vs Sleep Efficiency

In [25]:
y = sleep_df['Sleep efficiency'].values

X = sleep_df.drop(['Sleep efficiency','Smoking status_No','Smoking status_Yes'], axis=1).values

In [26]:
# Split data to training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [27]:
# Standard Scaler instance
scaler = StandardScaler()

# Fit Standard Scaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
# Neural Net model, nodes, layers

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 8)                 96        
                                                                 
 dense_7 (Dense)             (None, 5)                 45        
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 147 (588.00 Byte)
Trainable params: 147 (588.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
13/13 [==============================] - 1s 3ms/step - loss: 0.7703 - accuracy: 0.4217
Epoch 2/50
13/13 [==============================] - 0s 4ms/step - loss: 0.7330 - accuracy: 0.4798
Epoch 3/50
13/13 [==============================] - 0s 4ms/step - loss: 0.7035 - accuracy: 0.5404
Epoch 4/50
13/13 [==============================] - 0s 5ms/step - loss: 0.6806 - accuracy: 0.5884
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6604 - accuracy: 0.6162
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6419 - accuracy: 0.6465
Epoch 7/50
13/13 [==============================] - 0s 4ms/step - loss: 0.6237 - accuracy: 0.6944
Epoch 8/50
13/13 [==============================] - 0s 3ms/step - loss: 0.6057 - accuracy: 0.7071
Epoch 9/50
13/13 [==============================] - 0s 3ms/step - loss: 0.5869 - accuracy: 0.7197
Epoch 10/50
13/13 [==============================] - 0s 4ms/step - loss: 0.5684 - accuracy: 0.7576
Epoch 11/50
13/13 [

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5/5 - 0s - loss: 0.2488 - accuracy: 0.9242 - 208ms/epoch - 42ms/step
Loss: 0.2488008290529251, Accuracy: 0.9242424368858337


## Exercise frequency Vs Sleep efficiency

In [32]:
y = sleep_df['Sleep efficiency'].values

X = sleep_df.drop(['Sleep efficiency','Exercise frequency'], axis=1).values

In [33]:
# Split data to training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [34]:
# Standard Scaler instance
scaler = StandardScaler()

# Fit Standard Scaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
# Neural Net model, nodes, layers

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 8)                 104       
                                                                 
 dense_10 (Dense)            (None, 5)                 45        
                                                                 
 dense_11 (Dense)            (None, 1)                 6         
                                                                 
Total params: 155 (620.00 Byte)
Trainable params: 155 (620.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
13/13 [==============================] - 0s 4ms/step - loss: 0.2199 - accuracy: 0.9293
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 0.2166 - accuracy: 0.9318
Epoch 3/50
13/13 [==============================] - 0s 4ms/step - loss: 0.2147 - accuracy: 0.9293
Epoch 4/50
13/13 [==============================] - 0s 4ms/step - loss: 0.2129 - accuracy: 0.9318
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.2115 - accuracy: 0.9242
Epoch 6/50
13/13 [==============================] - 0s 4ms/step - loss: 0.2088 - accuracy: 0.9268
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.2070 - accuracy: 0.9268
Epoch 8/50
13/13 [==============================] - 0s 3ms/step - loss: 0.2052 - accuracy: 0.9293
Epoch 9/50
13/13 [==============================] - 0s 3ms/step - loss: 0.2034 - accuracy: 0.9343
Epoch 10/50
13/13 [==============================] - 0s 3ms/step - loss: 0.2018 - accuracy: 0.9343
Epoch 11/50
13/13 [

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5/5 - 0s - loss: 0.2368 - accuracy: 0.8636 - 43ms/epoch - 9ms/step
Loss: 0.23684339225292206, Accuracy: 0.8636363744735718
